In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os
import sys
import datetime
import matplotlib.pyplot as plt
import pyfolio as pf

sys.path.insert(0, '/mnt/afml/ml_finance/mlfinlab')
from mlfinlab.data_structures import imbalance_data_structures as imbar, standard_data_structures as bar
import mlfinlab as ml

sys.path.insert(0, '/mnt/afml/ml_finance/finance_ml')
from finance_ml import sampling, features

/mnt/afml/venv/lib/python3.5/site-packages/pyfolio/pos.py:28: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  ' to position notionals.'


In [4]:
def load_parq(fname):
    table = pq.read_table(fname)
    df = table.to_pandas()
    df = df.set_index('TIMESTAMP')
    ''' 중복된 index 제거, volume은 더해준다 '''
    df = df.sort_values(by='TIMESTAMP')  # 중복 데이터 무시
    df_v = df.groupby(df.index).sum()
    df = df.loc[~df.index.duplicated(keep='first')]
    df['V'] = df_v['V']
    df['DV'] = df_v['DV']
    return df

In [5]:
fname = 'dataset/TRADE_A233740_2018.parq'
df = load_parq(fname)

In [6]:
fname = 'dataset/TRADE_A233740_2018.csv'
bar_fname = 'dataset/DBAR_A233740_2018.csv'
if not os.path.exists(fname):
    df_csv = df.reset_index()[['TIMESTAMP', 'PRICE', 'V']]
    df_csv.columns = ['date_time', 'price', 'volume']
    df_csv['price'] = df_csv['price'].astype('float')
    df_csv.to_csv(fname, index=False)
    
if os.path.exists(bar_fname):
    dbar = pd.read_csv(bar_fname, index_col='date_time')
    dbar.index = pd.to_datetime(dbar.index)
else:
    dbar = bar.get_dollar_bars(fname, threshold=1e8)
    dbar.index = pd.to_datetime(dbar.index)
    dbar.to_csv(bar_fname)

In [7]:
# Compute daily volatility
daily_vol = ml.util.get_daily_vol(close=dbar['close'], lookback=50)

# Apply Symmetric CUSUM Filter and get timestamps for events
# Note: Only the CUSUM filter needs a point estimate for volatility
daily_vol_mean = daily_vol.rolling(10000).mean()
cusum_events = ml.filters.cusum_filter(dbar['close'], daily_vol_mean=daily_vol_mean)

# Compute vertical barrier
vertical_barriers = ml.labeling.add_vertical_barrier(t_events=cusum_events, close=dbar['close'], num_days=1)

Timestamp('2018-01-03 09:00:21.481000')


In [8]:
pt_sl = [1, 1]
min_ret = 0.005
triple_barrier_events = ml.labeling.get_events(close=dbar['close'],
                                               t_events=cusum_events,
                                               pt_sl=pt_sl,
                                               target=daily_vol,
                                               min_ret=min_ret,
                                               num_threads=3,
                                               vertical_barrier_times=vertical_barriers)

2019-06-07 16:12:17.435514 100.0% apply_pt_sl_on_t1 done after 0.1 minutes. Remaining 0.0 minutes.


In [9]:
labels_p = ml.labeling.get_bins(triple_barrier_events, dbar['close'])

In [10]:
raw_data = dbar.copy()

# Log Returns
raw_data['log_ret'] = np.log(raw_data['close']).diff()

# Momentum
raw_data['mom1'] = raw_data['close'].pct_change(periods=1)
raw_data['mom2'] = raw_data['close'].pct_change(periods=2)
raw_data['mom5'] = raw_data['close'].pct_change(periods=5)

# Volatility
raw_data['volatility_50'] = raw_data['log_ret'].rolling(window=50, min_periods=50, center=False).std()
raw_data['volatility_15'] = raw_data['log_ret'].rolling(window=15, min_periods=15, center=False).std()

# Serial Correlation (Takes about 4 minutes)
window_autocorr = 50

raw_data['autocorr_1'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=1), raw=False)
raw_data['autocorr_3'] = raw_data['log_ret'].rolling(window=window_autocorr, min_periods=window_autocorr, center=False).apply(lambda x: x.autocorr(lag=3), raw=False)

# Get the various log -t returns
raw_data['log_t1'] = raw_data['log_ret'].shift(1)
raw_data['log_t2'] = raw_data['log_ret'].shift(2)
raw_data['log_t5'] = raw_data['log_ret'].shift(5)

# Remove look ahead bias
raw_data = raw_data.shift(1)

In [24]:
# Get features at event dates
X = raw_data

# Drop unwanted columns
try:
    X.drop(['open', 'high', 'low', 'close', 'volume'], axis=1, inplace=True)
except Exception as e:
    print(e)

y = labels_p.loc[X.index,'bin']

"['open' 'high' 'low' 'close' 'volume'] not found in axis"


/mnt/afml/venv/lib/python3.5/site-packages/pandas/core/indexing.py:1017: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


## Ensemble Model

In [12]:
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, classification_report, confusion_matrix, accuracy_score
from sklearn.utils import resample
from sklearn.utils import shuffle

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [13]:
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp

from sklearn import datasets
from sklearn.metrics import roc_curve, auc
from itertools import cycle

In [26]:
y = y.dropna()
X = X.dropna()
com_idx = y.index.join(X.index).join(labels_p.index)
X = X.loc[com_idx]
y = y.loc[com_idx]
labels_p = labels_p.loc[com_idx]

In [ ]:
n_co_events = get_num_co_events(dbar['close'].index, t1, 2)
sample_weight = get_sample_tw(t1, n_co_events, triple_barrier_events.index)
labels_p['w'] = sample_weight

In [51]:
labels_p['t1'] = triple_barrier_events['t1']

## Feature Importance

In [43]:
from sklearn.metrics import log_loss, accuracy_score

from finance_ml.model_selection import PurgedKFold
from finance_ml.model_selection import cv_score


def feat_imp_MDI(forest, feat_names):
    imp_dict = {i:tree.feature_importances_ for i, tree in enumerate(forest.estimators_)}
    imp_df = pd.DataFrame.from_dict(imp_dict, orient='index')
    imp_df.columns = feat_names
    # 0 simply means not used for splitting
    imp_df = imp_df.replace(0, np.nan)
    imp = pd.concat({'mean': imp_df.mean(),
                     'std': imp_df.std() * np.sqrt(imp_df.shape[0])},
                    axis=1)
    imp /= imp['mean'].sum()
    return imp


def feat_imp_MDA(clf, X, y, n_splits, sample_weight, t1, pct_embargo, scoring='neg_log_loss'):
    if scoring not in ['neg_log_loss', 'accuracy']:
        raise Exception('wrong scoring method')
    cv_gen = PurgedKFold(n_splits=n_splits, t1=t1, pct_embargo=pct_embargo)
    index = np.arange(n_splits)
    scores = pd.Series(index=index)
    scores_perm = pd.DataFrame(index=index, columns=X.columns)
    for idx, (train, test) in zip(index, cv_gen.split(X=X)):
        X_train = X.iloc[train]
        y_train = y.iloc[train]
        w_train = sample_weight.iloc[train]
        X_test = X.iloc[test]
        y_test = y.iloc[test]
        w_test = sample_weight.iloc[test]
        clf_fit = clf.fit(X_train, y_train, sample_weight=w_train.values)
        if scoring == 'neg_log_loss':
            prob = clf_fit.predict_proba(X_test)
            scores.loc[idx] = -log_loss(y_test, prob, sample_weight=w_test.values,
                                     labels=clf_fit.classes_)
        else:
            pred = clf_fit.predict(X_test)
            scores.loc[idx] = accuracy_score(y_test,  pred, sample_weight=w_test.values)
        
        for col in X.columns:
            X_test_ = X_test.copy(deep=True)
            # Randomize certain feature to make it not effective
            np.random.shuffle(X_test_[col].values)
            if scoring == 'neg_log_loss':
                prob = clf_fit.predict_proba(X_test_)
                scores_perm.loc[idx, col] = -log_loss(y_test, prob, sample_weight=w_test.value,
                                                    labels=clf_fit.classes_)
            else:
                pred = clf_fit.predict(X_test_)
                scores_perm.loc[idx, col] = accuracy_score(y_test, pred, sample_weight=w_test.values)
    # (Original score) - (premutated score)
    imprv = (-scores_perm).add(scores, axis=0)
    # Relative to maximum improvement
    if scoring == 'neg_log_loss':
        max_imprv = -scores_perm
    else:
        max_imprv = 1. - scores_perm
    imp = imprv / max_imprv
    imp = pd.DataFrame({'mean': imp.mean(), 'std': imp.std() * np.sqrt(imp.shape[0])})
    return imp, scores.mean()
    

def aux_feat_imp_SFI(feat_names, clf, X, cont, scoring, cv_gen):
    imp = pd.DataFrame(columns=['mean', 'std'])
    for feat_name in feat_names:
        scores = cv_score(clf, X=X[[feat_name]], y=cont['bin'],
                          sample_weight=cont['w'],
                          scoring=scoring,
                          cv_gen=cv_gen)
        imp.loc[feat_name, 'mean'] = scores.mean()
        imp.loc[feat_name, 'std'] = scores.std() * np.sqrt(scores.shape[0])
    return imp

In [44]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier

from finance_ml.multiprocessing import mp_pandas_obj
from finance_ml.model_selection import PurgedKFold


def feat_importance(X, cont, clf=None, n_estimators=1000, n_splits=10, max_samples=1.,
                    num_threads=24, pct_embargo=0., scoring='accuracy',
                    method='SFI', min_w_leaf=0., **kwargs):
    n_jobs = (-1 if num_threads > 1 else 1)
    # Build classifiers
    if clf is None:
        base_clf = DecisionTreeClassifier(criterion='entropy', max_features=1,
                                          class_weight='balanced',
                                          min_weight_fraction_leaf=min_w_leaf)
        clf = BaggingClassifier(base_estimator=base_clf, n_estimators=n_estimators,
                                max_features=1., max_samples=max_samples,
                                oob_score=True, n_jobs=n_jobs)
    fit_clf = clf.fit(X, cont['bin'], sample_weight=cont['w'].values)
    if hasattr(fit_clf, 'oob_score_'):
        oob = fit_clf.oob_score_
    else:
        oob = None
    if method == 'MDI':
        imp = feat_imp_MDI(fit_clf, feat_names=X.columns)
        oos = cv_score(clf, X=X, y=cont['bin'], n_splits=n_splits,
                       sample_weight=cont['w'], t1=cont['t1'],
                       pct_embargo=pct_embargo, scoring=scoring).mean()
    elif method == 'MDA':
        imp, oos = feat_imp_MDA(clf, X=X, y=cont['bin'], n_splits=n_splits,
                                sample_weight=cont['w'], t1=cont['t1'],
                                pct_embargo=pct_embargo, scoring=scoring)
    elif method == 'SFI':
        cv_gen = PurgedKFold(n_splits=n_splits, t1=cont['t1'], pct_embargo=pct_embargo)
        oos = cv_score(clf, X=X, y=cont['bin'], sample_weight=cont['w'],
                       scoring=scoring, cv_gen=cv_gen)
        clf.n_jobs = 1
        imp = mp_pandas_obj(aux_feat_imp_SFI, ('feat_names', X.columns),
                            num_threads, clf=clf, X=X, cont=cont,
                            scoring=scoring, cv_gen=cv_gen)
    return imp, oob, oos

In [52]:
%%time
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(oob_score=True, n_estimators=100) 
imp_MDI, oob_MDI, oos_MDI = feat_importance(X, labels_p, clf=clf, method='MDI')
print(imp_MDI.head())
print(oob_MDI)
print(oos_MDI)

                   mean       std
log_ret        0.069718  0.126554
mom1           0.071654  0.113598
mom2           0.087857  0.131110
mom5           0.095547  0.135671
volatility_50  0.108679  0.133467
0.49322643074371025
0.4888130534628822
CPU times: user 1min 21s, sys: 8 ms, total: 1min 21s
Wall time: 1min 21s


In [56]:
%%time

clf = RandomForestClassifier(oob_score=True, n_estimators=100) 
imp_MDA, oob_MDA, oos_MDA = feat_importance(X, labels_p, clf=clf, method='MDA')
print(imp_MDA.head())
print(oob_MDA)
print(oos_MDA)

                   mean       std
log_ret       -0.030267  0.233171
mom1          -0.033689  0.199446
mom2          -0.006696  0.197762
mom5          -0.010387  0.164880
volatility_50 -0.016944  0.090630
0.49433231960188
0.4642213883516435
CPU times: user 1min 24s, sys: 40 ms, total: 1min 24s
Wall time: 1min 24s


In [57]:
%%time

clf = RandomForestClassifier(oob_score=True, n_estimators=100) 
imp_SFI, oob_SFI, oos_SFI = feat_importance(X, labels_p, clf=clf, method='SFI')
print(imp_SFI.head())
print(oob_SFI)
print(oos_SFI)

2019-06-11 20:34:39.232385 100.0% aux_feat_imp_SFI done after 8.54 minutes. Remaining 0.0 minutes.


                mean        std
autocorr_1    0.4434  0.0644523
autocorr_3  0.440873   0.113346
log_ret     0.451602    0.11345
log_t1      0.468481  0.0925326
log_t2      0.455479     0.1251
0.4946087918164225
[0.48266697 0.42317597 0.47497846 0.46732354 0.49861526 0.44876843
 0.47911657 0.51830764 0.42670918 0.46488995]
CPU times: user 1min 25s, sys: 1.03 s, total: 1min 26s
Wall time: 9min 57s


In [58]:
imp_SFI.sort_values('mean', ascending=False).index

Index(['log_t1', 'mom5', 'volatility_50', 'log_t2', 'mom1', 'log_ret',
       'volatility_15', 'mom2', 'log_t5', 'autocorr_1', 'autocorr_3'],
      dtype='object')

In [59]:
imp_MDI.sort_values('mean', ascending=False).index

Index(['volatility_50', 'volatility_15', 'autocorr_1', 'autocorr_3', 'mom5',
       'log_t5', 'log_t2', 'mom2', 'log_t1', 'mom1', 'log_ret'],
      dtype='object')

In [60]:
imp_MDA.sort_values('mean', ascending=False).index

Index(['log_t5', 'log_t1', 'mom2', 'mom5', 'autocorr_1', 'volatility_50',
       'log_ret', 'volatility_15', 'mom1', 'log_t2', 'autocorr_3'],
      dtype='object')